In [1]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.components.converters import TextFileToDocument
from haystack.components.converters import PyPDFToDocument
from haystack.document_stores.in_memory import InMemoryDocumentStore
from pathlib import Path

converterText = TextFileToDocument()
converterPDF = PyPDFToDocument()

# Meta Parameters

In [ ]:
if not 'basic_concepts' in locals():
    basic_concepts = ""

if not 'process_text' in locals():
    process_text = ""

if not 'process_name' in locals():
    process_name = ""

if not 'large_language_model' in locals():
    large_language_model = ""

if not 'prompt_name' in locals():
    prompt_name = ""

if not 'loop_limit' in locals():
    loop_limit = "20"

if not 'llm_url' in locals():
    llm_url = "http://localhost:11434"

# print("process_name:", process_name)
# print("process description: ", process_text)
# print("llm:" , large_language_model)
# print("prompt_name:", prompt_name)

# DCR - in-context learning / RGA

In [3]:
basicDCR = converterText.run(sources=[Path(basic_concepts)])

document_store = InMemoryDocumentStore()
document_store.write_documents(basicDCR['documents'])

Could not read .. Skipping it. Error: [Errno 21] Is a directory: '.'


0

# Prompt for Roles

In [ ]:
#Context
prompt_Role = """
[Context]
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Role = prompt_Role + """
{{ task }}
"""
task = """
Your task is to find all roles or actors for a DCR graph in the process description I will provide you with.
"""

# Output / Format
prompt_Role = prompt_Role + """
Return the roles in an array in JSON format, similar to: {"roles": [...,...,...]}. Do not forget the delimiter "," between the identified roles. Do not use any other delimiter. Do not add any other information to your answer. Do not include duplicate roles. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Role = prompt_Role + """
The process description in question is:
{{ process }}
"""

## Examples

In [5]:
prompt_Role = prompt_Role + """

Consider the following as a correct example: 
Example Description: "The process starts when the storage facility used by Puori sends a product sample to Ellipse. Ellipse then notifies Puori of the new sample. Before testing can begin, Puori then request a test of a sample through Ellipse. Ellipse must then test the product, as well as have additional testing done through Eurofarms."
Correctly identified roles: {"roles": ["Storage Facility", "Ellipse", "Puori"]}
"""

## Combine Role Pipeline

In [ ]:
retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_Role)
llm = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

role_pipeline = Pipeline()
role_pipeline.add_component("retriever", retriever)
role_pipeline.add_component("prompt_builder", prompt_builder)
role_pipeline.add_component("llm", llm)
role_pipeline.connect("retriever", "prompt_builder.documents")
role_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

# Prompt for Activities

In [ ]:
#Context
prompt_Activity = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Activity = prompt_Activity + """
Your task is to find all activities or events for a DCR graph in the process description I will provide you with and map them to the following roles: {{ roles }}.
"""

# Output / Format
prompt_Activity = prompt_Activity + """
Return the activitites as keys in key-value-pairs in JSON format, with their associated role as value, which should look like this: { {"activity": ..., "role": ...}, ...}. Do not forget the delimiter "," between the identified activities. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate activities. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Activity = prompt_Activity + """
The process description in question is:
{{ process }}
"""

## Examples

In [8]:
prompt_Activity = prompt_Activity + """

Consider the following as a correct example: 
Example Description: "Two weeks before the renting period is done, management needs to contact the seller to announce that the renting period is almost complete and ask the seller if they want an extension. However, it is also possible for sellers to contact management which then would not require management to contact them. If they do not want to extend, they will move out. If they would like an extension, management needs to check their revenue."
Correctly identified activities: [{"activity": "contact the seller to announce that the renting period is almost complete","role": "Management"},{"activity": "ask the seller if they want an extension","role": "Management"},{"activity": "contact management","role": "Seller"},{"activity": "do not want to extend","role": "Seller"},{"activity": "move out","role": "Seller"},{"activity": "like an extension","role": "Seller"},{"activity": "check their revenue","role": "Management"}]
"""
# [{"activity": brings in a defective computer, "role": "customer"},...]

## Combine Activity Pipeline

In [ ]:
retriever_Activity = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_Activity = PromptBuilder(template=prompt_Activity)
llm2 = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

activity_pipeline = Pipeline()
activity_pipeline.add_component("retriever_Activity", retriever_Activity)
activity_pipeline.add_component("prompt_builder_Activity", prompt_builder_Activity)
activity_pipeline.add_component("llm2", llm2)
activity_pipeline.connect("retriever_Activity", "prompt_builder_Activity.documents")
activity_pipeline.connect("prompt_builder_Activity", "llm2")

🚅 Components
  - retriever_Activity: InMemoryBM25Retriever
  - prompt_builder_Activity: PromptBuilder
  - llm2: OllamaGenerator
🛤️ Connections
  - retriever_Activity.documents -> prompt_builder_Activity.documents (List[Document])
  - prompt_builder_Activity.prompt -> llm2.prompt (str)

# Prompt for Rules

## Prompt for Condition

In [ ]:
#Context
prompt_Condition = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Condition = prompt_Condition + """
Your task is to find and define all Condition constraints between the activities for a DCR graph in the process description I will provide you with, based on the already identified activities: {{ activities }}.
The relation between the associated source activity and target activity must fall under the definition of a Condition in DCR. 
"""

# Output / Format
prompt_Condition = prompt_Condition + """
Return the DCR constraints as keys in key-value-pairs in JSON format, which should look like this: { {"constraint": "Condition", "source activity: ..., "target activity": ...}, ...}. The constraint should be one of the constraint types of DCR and the activities its associated source and target activity. Do not forget the delimiter "," between the identified constraints. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate constraints. Do not include the roles of the activities in the output. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
Remember that not all constraints between activities might be of type Condition, so you can also return an empty list.
"""

# Process Description
prompt_Condition = prompt_Condition + """
The process description in question is:
{{ process }}
"""

In [ ]:
prompt_Condition = prompt_Condition + """

Consider the following correct example:

Example for Condition: "Once they have completed everything, they can get the bike from the shop and the contract will be started."
Correctly identified Condition constraint: [{"constraint": "Condition", "source activity": "have completed everything", "target activity": "get the bike from the shop"},{"constraint": "Condition", "source activity": "have completed everything", "target activity": "the contract will be started"}]
"""

### Combine Condition Pipeline

In [ ]:
retriever_condition = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_Condition = PromptBuilder(template=prompt_Condition)
llm_cond = OllamaGenerator(model=model, url=llm_url, timeout=1200)

condition_pipeline = Pipeline()
condition_pipeline.add_component("retriever_condition", retriever_condition)
condition_pipeline.add_component("prompt_builder_Condition", prompt_builder_Condition)
condition_pipeline.add_component("llm_cond", llm_cond)
condition_pipeline.connect("retriever_condition", "prompt_builder_Condition.documents")
condition_pipeline.connect("prompt_builder_Condition", "llm_cond")

🚅 Components
  - prompt_builder_Rules: PromptBuilder
  - llm3: OllamaGenerator
🛤️ Connections
  - prompt_builder_Rules.prompt -> llm3.prompt (str)

## Prompt for Response

In [ ]:
#Context
prompt_Response = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Response = prompt_Response + """
Your task is to find and define all Response constraints between the activities for a DCR graph in the process description I will provide you with, based on the already identified activities: {{ activities }}.
The relation between the associated source activity and target activity must fall under the definition of a Response in DCR. 
"""

# Output / Format
prompt_Response = prompt_Response + """
Return the DCR constraints as keys in key-value-pairs in JSON format, which should look like this: { {"constraint": "Response", "source activity: ..., "target activity": ...}, ...}. The constraint should be one of the constraint types of DCR and the activities its associated source and target activity. Do not forget the delimiter "," between the identified constraints. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate constraints. Do not include the roles of the activities in the output. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
Remember that not all constraints between activities might be of type Response, so you can also return an empty list.
"""

# Process Description
prompt_Response = prompt_Response + """
The process description in question is:
{{ process }}
"""

In [ ]:
prompt_Response = prompt_Response + """

Consider the following correct example:

Example for Response: "If they do not want to extend, they will move out."
Correctly identified Response constraint: {"constraint": "Response", "source activity": "do not want to extend", "target activity": "move out"}
"""

### Combine Response Pipeline

In [ ]:
retriever_response = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_response = PromptBuilder(template=prompt_Response)
llm_resp = OllamaGenerator(model=model, url=llm_url, timeout=1200)

response_pipeline = Pipeline()
response_pipeline.add_component("retriever_response", retriever_response)
response_pipeline.add_component("prompt_builder_response", prompt_builder_response)
response_pipeline.add_component("llm_resp", llm_resp)
response_pipeline.connect("retriever_response", "prompt_builder_response.documents")
response_pipeline.connect("prompt_builder_response", "llm_resp")

🚅 Components
  - prompt_builder_Rules: PromptBuilder
  - llm3: OllamaGenerator
🛤️ Connections
  - prompt_builder_Rules.prompt -> llm3.prompt (str)

## Prompt for Include

In [ ]:
#Context
prompt_Include = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Include = prompt_Include + """
Your task is to find and define all Include constraints between the activities for a DCR graph in the process description I will provide you with, based on the already identified activities: {{ activities }}.
The relation between the associated source activity and target activity must fall under the definition of a Include in DCR. 
"""

# Output / Format
prompt_Include = prompt_Include + """
Return the DCR constraints as keys in key-value-pairs in JSON format, which should look like this: { {"constraint": "Include", "source activity: ..., "target activity": ...}, ...}. The constraint should be one of the constraint types of DCR and the activities its associated source and target activity. Do not forget the delimiter "," between the identified constraints. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate constraints. Do not include the roles of the activities in the output. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
Remember that not all constraints between activities might be of type Include, so you can also return an empty list.
"""

# Process Description
prompt_Include = prompt_Include + """
The process description in question is:
{{ process }}
"""

In [ ]:
prompt_Include = prompt_Include + """

Consider the following correct example:

Example for Include: "A customer must browse for products prior to adding a product to the basket, but they can access the basket upon entering the webshop without any prior conditions. In order to proceed to check-out at least one product must be in the basket."
Correctly identified Include constraint: {"constraint": "Include", "source activity": "adding a product to the basket", "target activity": "proceed to check-out"}
"""

### Combine Include Pipeline

In [ ]:
retriever_include = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_include = PromptBuilder(template=prompt_Include)
llm_include = OllamaGenerator(model=model, url=llm_url, timeout=1200)

include_pipeline = Pipeline()
include_pipeline.add_component("retriever_include", retriever_include)
include_pipeline.add_component("prompt_builder_include", prompt_builder_include)
include_pipeline.add_component("llm_include", llm_include)
include_pipeline.connect("retriever_include", "prompt_builder_include.documents")
include_pipeline.connect("prompt_builder_include", "llm_include")

🚅 Components
  - prompt_builder_Rules: PromptBuilder
  - llm3: OllamaGenerator
🛤️ Connections
  - prompt_builder_Rules.prompt -> llm3.prompt (str)

## Prompt for Exclude

In [ ]:
#Context
prompt_Exclude = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Exclude = prompt_Exclude + """
Your task is to find and define all Exclude constraints between the activities for a DCR graph in the process description I will provide you with, based on the already identified activities: {{ activities }}.
The relation between the associated source activity and target activity must fall under the definition of a Exclude in DCR.
"""

# Output / Format
prompt_Exclude = prompt_Exclude + """
Return the DCR constraints as keys in key-value-pairs in JSON format, which should look like this: { {"constraint": "Exclude", "source activity: ..., "target activity": ...}, ...}. The constraint should be one of the constraint types of DCR and the activities its associated source and target activity. Do not forget the delimiter "," between the identified constraints. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate constraints. Do not include the roles of the activities in the output. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
Remember that not all constraints between activities might be of type Exclude, so you can also return an empty list.
"""

# Process Description
prompt_Exclude = prompt_Exclude + """
The process description in question is:
{{ process }}
"""

In [ ]:
prompt_Exclude = prompt_Exclude + """

Consider the following correct example:

Example for Exclude: "Two weeks before the renting period is done, management needs to contact the seller to announce that the renting period is almost complete and ask the seller if they want an extension. However, it is also possible for sellers to contact management which then would not require management to contact them."
Correctly identified Exclude constraint: {"constraint": "Exclude", "source activity": "contact management", "target activity": "contact the seller to announce that the renting period is almost complete"}
"""

### Combine Exclude Pipeline

In [ ]:
retriever_exclude = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_exclude = PromptBuilder(template=prompt_Exclude)
llm_exclude = OllamaGenerator(model=model, url=llm_url, timeout=1200)

exclude_pipeline = Pipeline()
exclude_pipeline.add_component("retriever_exclude", retriever_exclude)
exclude_pipeline.add_component("prompt_builder_exclude", prompt_builder_exclude)
exclude_pipeline.add_component("llm_exclude", llm_exclude)
exclude_pipeline.connect("retriever_exclude", "prompt_builder_exclude.documents")
exclude_pipeline.connect("prompt_builder_exclude", "llm_exclude")

🚅 Components
  - prompt_builder_Rules: PromptBuilder
  - llm3: OllamaGenerator
🛤️ Connections
  - prompt_builder_Rules.prompt -> llm3.prompt (str)

## Prompt for Milestone

In [ ]:
#Context
prompt_Milestone = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Milestone = prompt_Milestone + """
Your task is to find and define all Milestone constraints between the activities for a DCR graph in the process description I will provide you with, based on the already identified activities: {{ activities }}.
The relation between the associated source activity and target activity must fall under the definition of a Milestone in DCR.
"""

# Output / Format
prompt_Milestone = prompt_Milestone + """
Return the DCR constraints as keys in key-value-pairs in JSON format, which should look like this: { {"constraint": "Milestone", "source activity: ..., "target activity": ...}, ...}. The constraint should be one of the constraint types of DCR and the activities its associated source and target activity. Do not forget the delimiter "," between the identified constraints. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate constraints. Do not include the roles of the activities in the output. Do not repeat the task description in your answer. Do neither describe nor explain your answer.
Remember that not all constraints between activities might be of type Milestone, so you can also return an empty list.
"""

# Process Description
prompt_Milestone = prompt_Milestone + """
The process description in question is:
{{ process }}
"""

In [ ]:
prompt_Milestone = prompt_Milestone + """

Consider the following correct example:

Example for Milestone: "After the design is approved by the product owner, Marketing may send the design to test groups for feedback. After the design is approved, if the design is not sent to test groups, Marketing must send the design to P&R."
Correctly identified Milestone constraint: {"constraint": "Milestone", "source activity": "approve the design", "target activity": "send the design to P&R"}
"""

### Combine Milestone Pipeline

In [ ]:
retriever_milestone = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_milestone = PromptBuilder(template=prompt_Milestone)
llm_mile = OllamaGenerator(model=model, url=llm_url, timeout=1200)

milestone_pipeline = Pipeline()
milestone_pipeline.add_component("retriever_milestone", retriever_milestone)
milestone_pipeline.add_component("prompt_builder_milestone", prompt_builder_milestone)
milestone_pipeline.add_component("llm_mile", llm_mile)
milestone_pipeline.connect("retriever_milestone", "prompt_builder_milestone.documents")
milestone_pipeline.connect("prompt_builder_milestone", "llm_mile")

🚅 Components
  - prompt_builder_Rules: PromptBuilder
  - llm3: OllamaGenerator
🛤️ Connections
  - prompt_builder_Rules.prompt -> llm3.prompt (str)

# Graph Generation

In [13]:
import json
import xml.etree.ElementTree as ET 
import uuid
from pathlib import Path
from datetime import datetime

## Run Prompt Pipeline

In [ ]:
def runPromptPipeline(): 
    start_time_role = datetime.now()
    response_Roles = role_pipeline.run(
        {
            "retriever": {"query": prompt_Role},
            "prompt_builder": {"task": task, "process": process_text},
        }
    )
    processing_time_role = datetime.now() - start_time_role
    processing_duration_role = processing_time_role.total_seconds()
    input_tokens_Role = response_Roles["llm"]["meta"][0]["prompt_eval_count"]
    output_tokens_Role = response_Roles["llm"]["meta"][0]["eval_count"]
    cleaned_roles = ''.join(response_Roles["llm"]["replies"])
    cleaned_roles = cleaned_roles.replace("'", '').replace("{", '').replace("}", '').replace("[", '').replace("]", '').replace(":", '').replace("\"roles\"", '').replace("\n",'').replace(" ", '').replace("\",\"",'\", \"').replace("\\",'')

    start_time_activity = datetime.now()
    response_Activities = activity_pipeline.run(
        {
            "retriever_Activity": {"query": prompt_Activity},
            "prompt_builder_Activity": {"process": process_text, "roles": cleaned_roles},
        }
    )
    processing_time_activity = datetime.now() - start_time_activity
    processing_duration_activity = processing_time_activity.total_seconds()
    input_tokens_Activity = response_Activities["llm2"]["meta"][0]["prompt_eval_count"]
    output_tokens_Activity = response_Activities["llm2"]["meta"][0]["eval_count"]
    cleaned_activities = ''.join(response_Activities["llm2"]["replies"])
    cleaned_activities = cleaned_activities.replace("'", '').replace("[", '').replace("]", '').replace("\"roles\"", '').replace("\n",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"activity','},{"activity').replace('} {"activity','},{"activity')

    start_time_condition = datetime.now()
    response_Condition = condition_pipeline.run(
        {
            "retriever_condition": {"query": prompt_Condition},
            "prompt_builder_Condition": {"process": process_text, "activities": cleaned_activities},
        }
    )
    processing_time_condition = datetime.now() - start_time_condition
    processing_duration_condition = processing_time_condition.total_seconds()
    input_tokens_condition = response_Condition["llm_cond"]["meta"][0]["prompt_eval_count"]
    output_tokens_condition = response_Condition["llm_cond"]["meta"][0]["eval_count"]
    cleaned_Condition = ''.join(response_Condition["llm_cond"]["replies"])
    cleaned_Condition = cleaned_Condition.replace("'", '').replace("[", '').replace("]", '').replace("\n",'').replace("  ",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"constraint','},{"constraint').replace('} {"constraint','},{"constraint')

    start_time_Response = datetime.now()
    response_Response = response_pipeline.run(
        {
            "retriever_response": {"query": prompt_Response},
            "prompt_builder_response": {"process": process_text, "activities": cleaned_activities},
        }
    )
    processing_time_Response = datetime.now() - start_time_Response
    processing_duration_response = processing_time_Response.total_seconds()
    input_tokens_response = response_Response["llm_resp"]["meta"][0]["prompt_eval_count"]
    output_tokens_response = response_Response["llm_resp"]["meta"][0]["eval_count"]
    cleaned_Response = ''.join(response_Response["llm_resp"]["replies"])
    cleaned_Response = cleaned_Response.replace("'", '').replace("[", '').replace("]", '').replace("\n",'').replace("  ",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"constraint','},{"constraint').replace('} {"constraint','},{"constraint')

    start_time_Include = datetime.now()
    response_Include = include_pipeline.run(
        {
            "retriever_include": {"query": prompt_Include},
            "prompt_builder_include": {"process": process_text, "activities": cleaned_activities},
        }
    )
    processing_time_Include = datetime.now() - start_time_Include
    processing_duration_include= processing_time_Include.total_seconds()
    input_tokens_include= response_Include["llm_include"]["meta"][0]["prompt_eval_count"]
    output_tokens_include= response_Include["llm_include"]["meta"][0]["eval_count"]
    cleaned_Include= ''.join(response_Include["llm_include"]["replies"])
    cleaned_Include= cleaned_Include.replace("'", '').replace("[", '').replace("]", '').replace("\n",'').replace("  ",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"constraint','},{"constraint').replace('} {"constraint','},{"constraint')

    start_time_Exclude = datetime.now()
    response_Exclude = exclude_pipeline.run(
        {
            "retriever_exclude": {"query": prompt_Exclude},
            "prompt_builder_exclude": {"process": process_text, "activities": cleaned_activities},
        }
    )
    processing_time_Exclude = datetime.now() - start_time_Exclude
    processing_duration_exclude= processing_time_Exclude.total_seconds()
    input_tokens_exclude= response_Exclude["llm_exclude"]["meta"][0]["prompt_eval_count"]
    output_tokens_exclude= response_Exclude["llm_exclude"]["meta"][0]["eval_count"]
    cleaned_Exclude= ''.join(response_Exclude["llm_exclude"]["replies"])
    cleaned_Exclude= cleaned_Exclude.replace("'", '').replace("[", '').replace("]", '').replace("\n",'').replace("  ",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"constraint','},{"constraint').replace('} {"constraint','},{"constraint')

    start_time_Milestone = datetime.now()
    response_Milestone = milestone_pipeline.run(
        {
            "retriever_milestone": {"query": prompt_Milestone},
            "prompt_builder_milestone": {"process": process_text, "activities": cleaned_activities},
        }
    )
    processing_time_Milestone = datetime.now() - start_time_Milestone
    processing_duration_milestone= processing_time_Milestone.total_seconds()
    input_tokens_milestone= response_Milestone["llm_mile"]["meta"][0]["prompt_eval_count"]
    output_tokens_milestone= response_Milestone["llm_mile"]["meta"][0]["eval_count"]
    cleaned_Milestone= ''.join(response_Milestone["llm_mile"]["replies"])
    cleaned_Milestone= cleaned_Milestone.replace("'", '').replace("[", '').replace("]", '').replace("\n",'').replace("  ",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"constraint','},{"constraint').replace('} {"constraint','},{"constraint')

    processing_duration_rule = processing_duration_condition + processing_duration_response + processing_duration_include + processing_duration_exclude + processing_duration_milestone
    input_tokens_Rule = (input_tokens_condition + input_tokens_response + input_tokens_include + input_tokens_exclude + input_tokens_milestone) / 5
    output_tokens_Rule = (output_tokens_condition + output_tokens_response + output_tokens_include + output_tokens_exclude + output_tokens_milestone) / 5
    cleaned_rules = cleaned_Condition + ", " + cleaned_Response + ", " + cleaned_Include + ", " + cleaned_Exclude + ", " + cleaned_Milestone

    return cleaned_roles, processing_duration_role, input_tokens_Role, output_tokens_Role, cleaned_activities, processing_duration_activity, input_tokens_Activity, output_tokens_Activity, cleaned_rules, processing_duration_rule, input_tokens_Rule, output_tokens_Rule

## Create JSON from Output

In [ ]:
def generateJSON(roles,activities,rules):
    rolesString = '"roles": [' + roles + ']'
    activitiesString = '"activities": [' + activities + ']'
    rulesString = '"constraints": [' + rules + ']'
    graphString = '{"DCRgraph": {' + rolesString + ', ' + activitiesString + ', ' + rulesString + '} }'
    
    # print(rolesString)
    # print(activitiesString)
    # print(rulesString)
    # print(graphString)

    graphJSON = json.loads(graphString)

    return graphJSON

## Generate DCR Editor XML

In [16]:
### the code for json_to_xml() was mostly generated using GPT-4o mini
def json_to_xml(json_obj):
    # Create the root element
    definitions = ET.Element("dcr:definitions", {
        "xmlns:dcr": "http://tk/schema/dcr",
        "xmlns:dcrDi": "http://tk/schema/dcrDi",
        "xmlns:dc": "http://www.omg.org/spec/DD/20100524/DC"
    })

    # Create dcrGraph element
    dcr_graph = ET.SubElement(definitions, "dcr:dcrGraph", id="dcrGraph")

    # Create events
    event_ids = {}
    for activity in json_obj["DCRgraph"]["activities"]:
        role = activity["role"]
        description = activity["activity"]
        event_id = f"Event_{uuid.uuid4().hex[:8]}"  # Generate a unique event ID
        event_ids[event_id] = activity  # Store the event ID with its activity for later use

        event = ET.SubElement(dcr_graph, "dcr:event", {
            "id": event_id,
            "role": role,
            "description": description,
            "included": "true",
            "executed": "false",
            "pending": "false",
            "enabled": "false"
        })

    # Create relations
    relation_ids = []
    for constraint in json_obj["DCRgraph"]["constraints"]:
        source_activity = constraint["source activity"]
        target_activity = constraint["target activity"]
        relation_id = f"Relation_{uuid.uuid4().hex[:8]}"  # Generate a unique relation ID

        #check whether constraint type matches DCR Editor XML types
        if constraint["constraint"] == "Condition":
            constraint_type = "condition"
        elif constraint["constraint"] == "Response":
            constraint_type = "response"
        elif constraint["constraint"] == "Exclude":
            constraint_type = "exclude"
        elif constraint["constraint"] == "Include":
            constraint_type = "include"
        elif constraint["constraint"] == "Milestone":
            constraint_type = "milestone"
        else:
            raise TypeError("Constraint Type was not identified correctly")
        
        # Find the corresponding event IDs
        source_event_id = next((eid for eid, act in event_ids.items() if act["activity"] == source_activity), None)
        target_event_id = next((eid for eid, act in event_ids.items() if act["activity"] == target_activity), None)

        if source_event_id and target_event_id:
            # Create the dcr:relation with a unique ID
            ET.SubElement(dcr_graph, "dcr:relation", {
                "id": relation_id,
                "type": constraint_type, 
                "sourceRef": source_event_id,
                "targetRef": target_event_id
            })
            relation_ids.append(relation_id)  # Store the relation ID for later use

    # Create dcrRootBoard element
    root_board = ET.SubElement(definitions, "dcrDi:dcrRootBoard", id="RootBoard")
    plane = ET.SubElement(root_board, "dcrDi:dcrPlane", id="Plane", boardElement="dcrGraph")

    # Create shapes for the diagram
    for event_id in event_ids.keys():
        shape = ET.SubElement(plane, "dcrDi:dcrShape", id=f"{event_id}_di", boardElement=event_id)
        # Example bounds, you can adjust the coordinates as needed
        ET.SubElement(shape, "dc:Bounds", x="640", y="230", width="130", height="150")

    # Create dcrDi:relation elements with unique IDs
    for relation_id in relation_ids:
        # Create a corresponding dcrDi:relation
        relation_di_id = f"{relation_id}_di"  # ID for dcrDi:relation
        relation_di = ET.SubElement(plane, "dcrDi:relation", id=relation_di_id, boardElement=relation_id)
        # Example waypoints, you can adjust the coordinates as needed
        ET.SubElement(relation_di, "dcrDi:waypoint", x="705", y="380")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="705", y="400")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="620", y="400")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="620", y="305")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="640", y="305")

    # Convert to string
    xml_str = ET.tostring(definitions, encoding='UTF-8', xml_declaration=True).decode('UTF-8')
    return xml_str



## Combined Graph Generation

In [ ]:
current_run = 0
generation_start_time = datetime.now()
errors = 0
role_times = []
role_input_token = []
role_output_token = []
activity_times = []
activity_input_token = []
activity_output_token = []
rule_times = []
rule_input_token = []
rule_output_token = []

while True:
    if current_run >= loop_limit:
        # safety mechanism to ensure termination if an unexpected issue is not handled
        # important to leave in when running everything automatically
        raise ValueError("Exceeded maximumg number of runs. Generation of Graph failed.")
    try:
        roles, role_time, role_input, role_output, activities, activity_time, activity_input, activity_output, rules, rule_time, rule_input, rule_output = runPromptPipeline()
        role_times.append(role_time)
        role_input_token.append(role_input)
        role_output_token.append(role_output)
        activity_times.append(activity_time)
        activity_input_token.append(activity_input)
        activity_output_token.append(activity_output)
        rule_times.append(rule_time)
        rule_input_token.append(rule_input)
        rule_output_token.append(rule_output)
        graphJSON = generateJSON(roles,activities,rules)
        print("Parsing to JSON successful")
        xml = json_to_xml(graphJSON)
        print(f"XML Generation successful ({process_name},Pipeline_A2_{prompt_name})")
        print("xml:", xml)
        print("\n")
        break
    except Exception as e:
        current_run += 1
        errors += 1
        print(f"Graph Generation failed with error ({process_name},Pipeline_A2_{prompt_name})")
        print(e, flush=True)
        runs_left = loop_limit - current_run
        print("Iterations left: " + str(runs_left))
        print("\n")

generation_finished_time = datetime.now() - generation_start_time
generation_duration = generation_finished_time.total_seconds()

Graph Generation failed with error (,Pipeline_C1_)
1 validation error for GenerateRequest
model
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
Iterations left: 19


Graph Generation failed with error (,Pipeline_C1_)
1 validation error for GenerateRequest
model
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
Iterations left: 18


Graph Generation failed with error (,Pipeline_C1_)
1 validation error for GenerateRequest
model
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
Iterations left: 17


Graph Generation failed with error (,Pipeline_C1_)
1 validation error for GenerateRequest
mode

ValueError: Exceeded maximumg number of runs. Generation of Graph failed.

## Save Time to Results

In [ ]:
new_times = f"{process_name};{generation_duration};{role_times};{role_input_token};{role_output_token};{activity_times};{activity_input_token};{activity_output_token};{rule_times};{rule_input_token};{rule_output_token};{errors}\n"

folder = Path('./6_Time_Analysis') / "data" / large_language_model / "Pipeline_A2"
folder.mkdir(parents=True, exist_ok=True) 

file_name = f"{prompt_name}.txt"
file_path = folder / file_name 

with open(file_path, 'a+') as file:
    file.write(new_times)

## Save XML to Results

In [ ]:
folder = Path(result_xml_path) / large_language_model / "Pipeline_A2" / prompt_name
folder.mkdir(parents=True, exist_ok=True) 

file_name = f"{process_name}_{large_language_model}_{prompt_name}_{datetime.now().date()}_{datetime.now().time()}.xml"
file_path = folder / file_name 

with open(file_path, 'w') as result_destination:
    result_destination.write(xml)